In [1]:
import numpy as np
import pandas as pd
import statistics as st
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sparse_filtering import SparseFiltering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
file = '/data/bioprotean/ABA/MEMMAP/genes_list/finalgenes_mask.npy'
mask = np.load(file)
mean = mask.mean()
imputed = np.where(mask < 0, mean, mask)
print(mask.shape)
print('gap')
print(imputed)
print(imputed.shape)

(63113, 2941)
gap
[[3.5294919e+00 3.5294919e+00 3.5294919e+00 ... 3.5294919e+00
  3.5294919e+00 3.5294919e+00]
 [3.5294919e+00 3.5294919e+00 3.5294919e+00 ... 3.5294919e+00
  3.5294919e+00 3.5294919e+00]
 [3.5294919e+00 3.5294919e+00 3.5294919e+00 ... 3.5294919e+00
  3.5294919e+00 3.5294919e+00]
 ...
 [1.0158209e+00 5.6641757e-02 6.0131657e-01 ... 5.8417840e+00
  7.0080962e-03 3.9926261e-02]
 [4.3311420e-01 2.0019943e-02 7.3028982e-01 ... 6.6745105e+00
  2.2064042e-03 6.3012719e-02]
 [1.1118312e+00 2.1597330e-02 5.2785075e-01 ... 5.5462308e+00
  1.2890357e-02 3.5294919e+00]]
(63113, 2941)


In [7]:
'''Running Sparse Filtering'''

# Inputting the number of features and directory path
dir_path = '/home/nraghu/Naren/SFT/Imputation/Features/'
numbers_list = range(50,601,50)
# numbers_list = [100]

# Loading the voxel * gene matrix
X = imputed

In [8]:
for n in numbers_list:
    estimator = SparseFiltering(n_features = n, maxfun = 200, iprint = 10)
    features = estimator.fit_transform(X)
    weights = estimator.w_
    
    # Saving features and weights
    np.save(dir_path + str(n) + '_features.npy', features)
    np.save(dir_path + str(n) + '_weights.npy', weights)

KeyboardInterrupt: 

In [9]:
# Inputting directories
features_path = '/home/nraghu/Naren/SFT/Imputation/Features/'
save_path = '/home/nraghu/Naren/SFT/Imputation/clusters_mask/'

# List of feature numbers
# features_list = range(100,101,50)
features_list = [100]

# List of cluster numbers
# numbers_list = list(range(1,50))
# extra_list = list(range(50,551,50))
# add_number = 594
# numbers_list.extend(extra_list)
# numbers_list.append(add_number)
numbers_list = [100]

for n_features in features_list:
    # Loading features
    features = np.load(features_path + str(n_features) + '_features.npy')
    
    # Running K-means
    for n_clusters in numbers_list:
        kmeans = KMeans(n_clusters = n_clusters, random_state = 0)
        kmeans.fit_predict(X)
        labels = kmeans.labels_
        
        # Saving the labels
        np.save(save_path + str(n_features) + 'features_' + str(n_clusters) + '_clusters.npy', labels)

In [3]:
def MI_score_df_maker(reference, clusters_dir, mode = 'AMI', csv_path = None):
    '''
    Calculates Mutual Information score for the labels and the reference.
    Saves them in a Pandas DataFrame.
    
    Arguments:
        reference: arr
            Array of the reference labels.
        
        clusters_dir: str
            The path to the clusters folder.
        
        mode : {'AMI', 'NMI'}
            The Mutual Information method to compute.
        
        csv_path: str, default = None
            The path to save the DataFrame as a csv file.
    
    Returns:
        score_df: DataFrame
            Pandas dataframe of scores and K range.
    '''
    
    # Making a dataframe of AMI scores
    score_df = pd.DataFrame(columns=['K','AMI'])
    
    # List of K values
    numbers_list = list(range(1,50))
    extra_list = list(range(50,551,50))
    numbers_list.extend(extra_list)
    
    # The list of scores
    MI_list = []
    
    for i in numbers_list:
        # Loading the labels
        cluster_path = clusters_dir + str(i) + 'clusters.npy'
        # cluster_path = clusters_dir + str(i) + '.npy'
        #cluster_path = clusters_dir + str(i) + 'c.npy'
        cluster = np.load(cluster_path).flatten()
        
        if mode == 'AMI':
            MI_list.append(AMI(reference, cluster))
        
        elif mode == 'NMI':
            MI_list.append(NMI(reference, cluster))
    
    # Adding values to DataFrame
    score_df['K'] = numbers_list
    score_df['AMI'] = MI_list
    
    # Saving to CSV
    if csv_path != None:
        score_df.to_csv(csv_path)
    
    return score_df

In [4]:
'''For SFT clusters'''

# Loading the Allen Reference Atlas
allen_path = '/data/bioprotean/ABA/Template/allen_annot200_mask.npy'
reference = np.load(allen_path)

# Inputting number of features
n_features_list = range(100,101,50)

# Save to dataframe and file
for n_features in n_features_list:
    # Path to clusters directory
    clusters_dir = '/home/nraghu/Naren/SFT/Imputation/clusters_mask/'+str(n_features)+'features_'
    
    # Save to dataframe and file
    score_df = MI_score_df_maker(reference, clusters_dir)
    score_df.to_csv('/home/nraghu/Naren/SFT/MI_imp/rc'+str(n_features)+'f_MI_scores.csv')

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float32'), dtype('<U1')) -> None